# Urban Water Model - Cell Flow Analysis

This notebook provides interactive cell selection and flow visualization tools for analyzing water flows at the individual cell level.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

from ipywidgets import Button, Output
import ipywidgets as widgets
import plotly.graph_objects as go

import holoviews as hv
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

from duwcm.read_data import read_data
from duwcm.forcing import read_forcing
from duwcm.water_model import UrbanWaterModel
from duwcm.water_balance import run_water_balance
from duwcm.utils import load_config
from duwcm.functions import select_cells
from duwcm.postprocess import extract_local_results
from duwcm.diagnostics import DiagnosticTracker

from duwcm.viz import (
    interactive_cell_selection,
    create_flows,
    create_cell_flows,
    create_map_base
)

## Configuration

Set up the configuration and load required data.

In [ ]:
config = load_config('.', 'default', 'config.yaml')

print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

geo_file = Path(config.input_directory) / Path(config.files.geo)
background_file = Path(config.geodata_directory) / config.files.background_shapefile

_, _, _, _, _, flow_paths = read_data(config)

# Initialize diagnostic tracker
tracker = DiagnosticTracker()

# Call the interactive selection function
selected_cells = interactive_cell_selection(config, geo_file, background_file, flow_paths)

## Run Simulation

Execute the urban water model simulation for selected cells.

In [ ]:
# Read and prepare model data
model_params, reuse_settings, demand_data, soil_data, et_data, flow_paths = read_data(config)
forcing_data = read_forcing(config)

# Filter selected cells if specified
if hasattr(config.grid, 'selected_cells'):
    model_params, flow_paths = select_cells(model_params, flow_paths, config.grid.selected_cells)

# Initialize and run model
model = UrbanWaterModel(
    params=model_params,
    path=flow_paths,
    soil_data=soil_data,
    et_data=et_data,
    demand_settings=demand_data,
    reuse_settings=reuse_settings,
    direction=config.grid.direction
)

results = run_water_balance(model, forcing_data, tracker=tracker)

## Cell Flow Analysis

Analyze internal and external flows for selected cells.

In [ ]:
# Create cell selector dropdown
cell_selector = widgets.Dropdown(
    options=sorted(config.grid.selected_cells) if hasattr(config.grid, 'selected_cells') else sorted(model_params.keys()),
    description='Cell ID:'
)

# Create output area for plots
plot_output = widgets.Output()

def on_cell_change(change):
    with plot_output:
        plot_output.clear_output()
        internal_fig, external_fig = create_cell_flows(change.new, tracker)
        internal_fig.show()
        external_fig.show()

cell_selector.observe(on_cell_change, names='value')

# Display widgets
display(cell_selector)
display(plot_output)

# Initial plot
if cell_selector.options:
    with plot_output:
        internal_fig, external_fig = create_cell_flows(cell_selector.value, tracker)
        internal_fig.show()
        external_fig.show()